In [19]:
import numpy as np 
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.optimizers import Adam

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [20]:
data_path = 'Outputs/total_array.csv'
raw_data = np.loadtxt(data_path, delimiter=',', skiprows=0)

In [21]:
print(raw_data.shape)
num_sample = raw_data.shape[0]
num_time_steps = 10
x_dim = 201
y_dim = 1

(2121, 211)


In [4]:
def get_label(stock_price, pre_stock_price):
    
    if abs(stock_price - pre_stock_price)/pre_stock_price <= 0.00016:
        label = 0
    elif (stock_price - pre_stock_price)/pre_stock_price > 0.00016:
        label = 1
    else:
        label = 2
        
    return label

In [22]:
scaler = MinMaxScaler(feature_range=(0, 1))
raw_data[:, 1: 11] = scaler.fit_transform(raw_data[:, 1: 11])

In [23]:
X = []
y = []

for sample in range(num_sample):
    data_input = raw_data[sample, :]
    stock_labels = raw_data[sample, 0: num_time_steps + 1]

    y.append(stock_labels[0])
    X.append(data_input[1:])

In [24]:
scaler_y = MinMaxScaler(feature_range=(0, 1))
y = scaler_y.fit_transform(np.array(y).reshape((len(y), 1)))

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [26]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [27]:
print(X_train.shape)
print(X_test.shape)
#print(y_train.shape)
print(y_train)
y_binary = to_categorical(y_train)

(1696, 210)
(425, 210)
[[0.42404778]
 [0.32629655]
 [0.77691383]
 ...
 [0.46114742]
 [0.65388275]
 [0.87636792]]


In [32]:
n_a = 128 # number of dimensions for the hidden state of each LSTM cell.

In [33]:
n_values = 201 # number of input values
reshapor = Reshape((1, n_values))                       
LSTM_cell = LSTM(n_a, return_state = True)  
densor = Dense(1)

In [28]:
model = Sequential()
model.add(Dense(64, input_dim=210, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

In [47]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 64)                412672    
_________________________________________________________________
dense_18 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_19 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 17        
Total params: 415,297
Trainable params: 415,297
Non-trainable params: 0
_________________________________________________________________


In [29]:
train_f1 = []
test_f1 = []

In [29]:
model.fit(X_train, y_train, epochs=50, batch_size=1)

Epoch 1/50
1696/1696 [==============================] - 6s 3ms/step - loss: 0.4679 - mean_absolute_error: 0.4609
Epoch 2/50
1696/1696 [==============================] - 5s 3ms/step - loss: 0.0620 - mean_absolute_error: 0.1854
Epoch 3/50
1696/1696 [==============================] - 5s 3ms/step - loss: 0.0262 - mean_absolute_error: 0.1248
Epoch 4/50
1696/1696 [==============================] - 5s 3ms/step - loss: 0.0183 - mean_absolute_error: 0.1019
Epoch 5/50
1696/1696 [==============================] - 5s 3ms/step - loss: 0.0291 - mean_absolute_error: 0.1168
Epoch 6/50
1696/1696 [==============================] - 5s 3ms/step - loss: 0.0136 - mean_absolute_error: 0.0840
Epoch 7/50
1696/1696 [==============================] - 5s 3ms/step - loss: 0.0122 - mean_absolute_error: 0.0755
Epoch 8/50
1696/1696 [==============================] - 5s 3ms/step - loss: 0.0040 - mean_absolute_error: 0.0466
Epoch 9/50
1696/1696 [==============================] - 5s 3ms/step - loss: 0.0046 - mean_absolu

In [30]:
for epoch in range(1, 25):
    model.fit(X_train, y_binary, epochs=1, batch_size=1)
    trainPredict = model.predict(X_train, batch_size=1)
    train_f1.append(f1_score(y_train, np.argmax(trainPredict, axis=1), average='micro'))
    
    testPredict = model.predict(X_test, batch_size=1)
    test_f1.append(f1_score(y_test, np.argmax(testPredict, axis=1), average='micro'))

Epoch 1/1
1696/1696 [==============================] - 7s 4ms/step - loss: 1.0078 - acc: 0.5289
Epoch 1/1
1696/1696 [==============================] - 7s 4ms/step - loss: 0.8536 - acc: 0.5879
Epoch 1/1
1696/1696 [==============================] - 7s 4ms/step - loss: 0.4197 - acc: 0.8261
Epoch 1/1
1696/1696 [==============================] - 7s 4ms/step - loss: 0.1490 - acc: 0.9493
Epoch 1/1
1696/1696 [==============================] - 7s 4ms/step - loss: 0.0766 - acc: 0.9752
Epoch 1/1
1696/1696 [==============================] - 7s 4ms/step - loss: 0.0736 - acc: 0.9752
Epoch 1/1
1696/1696 [==============================] - 7s 4ms/step - loss: 0.0523 - acc: 0.9776
Epoch 1/1
1696/1696 [==============================] - 7s 4ms/step - loss: 0.0390 - acc: 0.9847
Epoch 1/1
1696/1696 [==============================] - 7s 4ms/step - loss: 0.0379 - acc: 0.9870
Epoch 1/1
1696/1696 [==============================] - 7s 4ms/step - loss: 0.0571 - acc: 0.9794
Epoch 1/1
1696/1696 [===================

In [30]:
trainPredict = model.predict(X_train, batch_size=1)

In [31]:
trainPredict = scaler_y.inverse_transform(trainPredict)
y_train = scaler_y.inverse_transform(y_train)

In [32]:
for idx, pred in enumerate(trainPredict):
    print(pred, y_train[idx])

[2480.113] [2477.31]
[2398.6702] [2399.24]
[2760.3054] [2759.13]
[2276.4333] [2272.23]
[2411.6006] [2397.71]
[2717.7168] [2713.19]
[2713.294] [2711.51]
[2259.6423] [2267.56]
[2763.761] [2771.34]
[2838.0999] [2841.92]
[2427.7927] [2431.52]
[2483.183] [2500.83]
[2391.8704] [2390.19]
[2734.5212] [2723.82]
[2509.0583] [2505.86]
[2877.897] [2873.69]
[2848.545] [2844.07]
[2183.8572] [2176.26]
[2737.359] [2738.7]
[2452.2007] [2445.35]
[2745.4883] [2729.17]
[2643.282] [2648.77]
[2510.7727] [2506.68]
[2643.6384] [2633.45]
[2806.9023] [2794.74]
[2882.2563] [2898.5]
[2845.4373] [2855.14]
[2936.27] [2933.58]
[2793.969] [2783.38]
[2889.6685] [2886.95]
[2569.742] [2563.69]
[2556.4858] [2566.42]
[2556.3896] [2562.64]
[2717.878] [2723.78]
[2381.8987] [2382.86]
[2768.5981] [2770.67]
[2679.8364] [2666.4]
[2594.8748] [2603.48]
[2851.2961] [2858.59]
[2840.855] [2815.93]
[2514.5088] [2506.8]
[2839.6687] [2840.13]
[2544.1035] [2545.24]
[2441.2253] [2446.21]
[2803.4343] [2801.15]
[2523.777] [2536.91]
[2662.7

[2649.5378] [2668.76]
[2925.3315] [2907.26]
[2197.7278] [2185.62]
[2468.0713] [2462.4]
[2833.4065] [2807.19]
[2834.96] [2840.4]
[2415.3772] [2393.43]
[2842.8787] [2850.84]
[2654.6272] [2657.86]
[2860.1304] [2864.78]
[2622.4116] [2625.13]
[2433.786] [2441.43]
[2311.418] [2296.98]
[2452.7664] [2446.29]
[2696.933] [2677.26]
[2900.6648] [2899.78]
[2523.2307] [2510.61]
[2672.6255] [2668.55]
[2621.8445] [2644.17]
[2502.147] [2495.74]
[2911.4614] [2909.96]
[2457.3828] [2453.24]
[2671.4788] [2671.48]
[2724.7214] [2727.84]
[2718.8918] [2716.79]
[2879.057] [2860.67]
[2673.7056] [2676.29]
[2813.2717] [2806.21]
[2376.7363] [2361.89]
[2771.2278] [2749.34]
[2915.4548] [2906.67]
[2501.632] [2497.22]
[2565.387] [2564.41]
[2444.7695] [2444.41]
[2905.0933] [2899.26]
[2650.1316] [2655.24]
[2470.0254] [2468.56]
[2684.505] [2690.92]
[2447.784] [2443.55]
[2738.4954] [2731.29]
[2516.5295] [2514.96]
[2473.5413] [2457.7]
[2198.9592] [2198.92]
[2765.2612] [2765.62]
[2281.3723] [2272.47]
[2286.8245] [2274.76]
[2

[2280.1301] [2269.76]
[2745.352] [2736.65]
[2709.6816] [2708.44]
[2793.0388] [2786.5]
[2575.8179] [2569.72]
[2248.3076] [2255.87]
[2209.693] [2208.42]
[2502.5874] [2495.62]
[2753.7847] [2781.05]
[2439.2676] [2428.9]
[2466.8008] [2468.23]
[2876.903] [2870.29]
[2647.5784] [2640.81]
[2742.1953] [2742.04]
[2780.5476] [2782.25]
[2752.1511] [2745.51]
[2676.303] [2677.44]
[2404.2573] [2394.18]
[2553.324] [2550.87]
[2687.1626] [2688.59]
[2447.2268] [2439.47]
[2767.4546] [2762.34]
[2759.1885] [2755.17]
[2719.9785] [2693.46]
[2858.2617] [2877.3]
[2570.6433] [2561.68]
[2892.6157] [2900.85]
[2352.695] [2351.24]
[2276.757] [2270.22]
[2396.7444] [2388.53]
[2416.2751] [2430.11]
[2264.0547] [2267.8]
[2438.7666] [2434.81]
[2310.0925] [2307.8]
[2715.8755] [2682.78]
[2551.0964] [2548.94]
[2881.5042] [2880.36]
[2914.526] [2906.55]
[2774.0127] [2759.13]
[2491.656] [2495.13]
[2893.3914] [2903.33]
[2437.7659] [2413.16]
[2375.7646] [2386.1]
[2734.2312] [2728.61]
[2277.0776] [2272.48]
[2471.062] [2462.69]
[270

[2697.5176] [2707.85]
[2744.1228] [2729.24]
[2446.9802] [2438.91]
[2928.3938] [2927.38]
[2655.6528] [2652.42]
[2781.9272] [2793.38]
[2796.4072] [2797.18]
[2758.582] [2746.31]
[2721.2961] [2713.82]
[2742.553] [2745.81]
[2647.2563] [2652.87]
[2541.4875] [2551.81]
[2433.7944] [2438.17]
[2295.5562] [2293.02]
[2185.6394] [2172.7]
[2930.5022] [2934.22]
[2843.1829] [2838.58]
[2730.613] [2732.05]
[2666.6306] [2656.38]
[2732.3186] [2725.22]
[2559.8823] [2565.37]
[2472.443] [2476.97]
[2468.5293] [2467.39]
[2823.1787] [2820.21]
[2379.5747] [2383.12]
[2702.2974] [2684.48]
[2714.6643] [2715.75]
[2761.859] [2756.77]
[2707.7866] [2715.96]
[2764.068] [2760.85]
[2851.5315] [2867.46]
[2303.4497] [2294.5]
[2662.9072] [2657.97]
[2755.9565] [2767.53]
[2774.917] [2770.36]
[2744.4404] [2741.93]
[2604.8223] [2595.86]
[2721.3408] [2724.45]
[2886.306] [2889.32]
[2495.7974] [2498.57]
[2593.5864] [2575.9]
[2566.21] [2554.63]
[2857.357] [2862.7]
[2761.8518] [2768.74]
[2278.1646] [2272.4]
[2731.883] [2728.05]
[2857

In [54]:
print(confusion_matrix(y_train, np.argmax(trainPredict, axis=1)))
print(f1_score(y_train, np.argmax(trainPredict, axis=1), average='micro'))

ValueError: Classification metrics can't handle a mix of continuous and binary targets

In [33]:
print(r2_score(y_train, trainPredict))
print(mean_squared_error(y_train, trainPredict))
print(mean_absolute_error(y_train, trainPredict))

0.9975192069656476
90.24087326802314
7.07512848476194


In [34]:
testPredict = model.predict(X_test, batch_size=1)

In [35]:
testPredict = scaler_y.inverse_transform(testPredict)
y_test = scaler_y.inverse_transform(y_test)

In [39]:
np.savetxt('Outputs/test.csv', y_test, delimiter=",")

In [17]:
print(confusion_matrix(y_test, np.argmax(testPredict, axis=1)))
print(f1_score(y_test, np.argmax(testPredict, axis=1), average='micro'))

[[  0   3  48]
 [ 16   9 130]
 [ 36  11 172]]
0.4258823529411765


In [36]:
print(r2_score(y_test, testPredict))
print(mean_squared_error(y_test, testPredict))
print(mean_absolute_error(y_test, testPredict))

0.9970093882961601
105.26037109125457
7.82133908547794
